In [2]:
!openai --version

openai 1.44.0


In [1]:
!pip install openai --upgrade

  Using cached openai-1.44.0-py3-none-any.whl.metadata (22 kB)
Using cached openai-1.44.0-py3-none-any.whl (367 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.28.0
    Uninstalling openai-1.28.0:
      Successfully uninstalled openai-1.28.0


In [3]:
import os
directory_folder = "data"

pdf_files = [os.path.join(directory_folder,file_name) for file_name in os.listdir(directory_folder) \
          if os.path.isfile(os.path.join(directory_folder,file_name))]



In [4]:
# Dictionary to store documents with filename as keys

from langchain.document_loaders import PyPDFLoader

documents_dict = {}
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents = loader.load()

    doc_key = pdf_file.split("\\")[1]
    documents_dict[doc_key] = documents

In [5]:
from langchain.embeddings import OpenAIEmbeddings
os.environ['OPENAI_API_KEY'] = "***********"


In [6]:
embedding_model = OpenAIEmbeddings()

C:\Users\alams\AppData\Local\Temp\ipykernel_8940\909470085.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding_model = OpenAIEmbeddings()


In [7]:
from langchain.schema import Document
documents = []

for filename, docs in documents_dict.items():
    content = "".join([doc.page_content for doc in docs])
    # embeddings = [embedding_model.embed_query(text) for text in texts]

    documents.append(Document(page_content=content, metadata={"source": filename}))
    # embedding_dict[filename] = embeddings
    # embe

In [8]:

PINECONE_API_KEY = "**********"
os.environ['PINECONE_API_KEY']=PINECONE_API_KEY

In [9]:
!openai --version

openai 1.44.0


In [10]:
from langchain_pinecone import PineconeVectorStore


vector_store = PineconeVectorStore.from_documents(documents,embedding_model, index_name = "pdf-program-index" )


c:\Users\alams\anaconda3\envs\openai_assitannt_chatbot_env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [42]:

# vector_store.delete(delete_all=True)

In [11]:
from langchain.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)
vectore_store_retriever = vector_store.as_retriever()

In [12]:
vectore_store_retriever.search_kwargs = {"k":1}
bm25_retriever.k = 1

In [13]:
vectore_store_retriever.invoke("What is SNAP")

[Document(metadata={'source': 'SNAP.pdf'}, page_content='Program Name: SNAP  \nFood Stamps was renamed the Supplemental Nutrition Assistance Program (SNAP) in 2008  \nAgency: Massachusetts Department of Transitional Assistance (DTA)  \nDescription: SNAP is a federal nutrition program administered by the Department of Transitional \nAssistance (DTA) for Massachusetts residents. The benefits can be used in food stores but not \nrestaurants to purchase food items.  \nEligibility:  \nHousehold gross income not to exceed 130% of Federal Poverty Level and household net income not \nto exceed 100% FPL to be eligible.   \nDeductions and some exceptions apply to income determination. Asset limits and work requirements \nalso apply.  \nSNAP eligibility is based on who is in the household, income and certain expenses. “Eligibility ” means \nwho can and can’t get the benefit.  \nIf you aren ’t sure if you are eligible, apply! The agencywill talk to you and make a decision based on \nthe rules. If 

In [14]:
from langchain.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(retrievers=[vectore_store_retriever, bm25_retriever], weights=[0.3, 0.7])

In [15]:
ensemble_retriever.invoke("How to apply for SNAP?")

[Document(metadata={'source': 'SNAP.pdf'}, page_content='Program Name: SNAP  \nFood Stamps was renamed the Supplemental Nutrition Assistance Program (SNAP) in 2008  \nAgency: Massachusetts Department of Transitional Assistance (DTA)  \nDescription: SNAP is a federal nutrition program administered by the Department of Transitional \nAssistance (DTA) for Massachusetts residents. The benefits can be used in food stores but not \nrestaurants to purchase food items.  \nEligibility:  \nHousehold gross income not to exceed 130% of Federal Poverty Level and household net income not \nto exceed 100% FPL to be eligible.   \nDeductions and some exceptions apply to income determination. Asset limits and work requirements \nalso apply.  \nSNAP eligibility is based on who is in the household, income and certain expenses. “Eligibility ” means \nwho can and can’t get the benefit.  \nIf you aren ’t sure if you are eligible, apply! The agencywill talk to you and make a decision based on \nthe rules. If 

In [71]:
template = """
You are an assistant tasked with generating responses that include relevant hyperlinks when providing information. Based on the following user input, ensure that the response includes a hyperlink to any relevant forms, documents, or resources.

Question: {question}

Generate a response that:

1. Ensure that the response includes a hyperlink to relevant forms or resources when applicable.
2. Is kind, inviting, and supportive.
3. If retrieved context source are `SNAP.PDF`or `WIC.pdf, and `Lifeline_Program.pdf` then in genearted response please include the link. 
4.. If a customer mentions filling out one form from `SNAP.PDF`or `WIC.pdf, and `Lifeline_Program.pdf, encourage them to fill out the other forms as well by providing their link


Context: {context}
Answer:
"""

In [72]:
from langchain_openai import ChatOpenAI


In [36]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [73]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)
output_parser=StrOutputParser()

In [74]:
rag_chain = (
    {"context": vectore_store_retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [75]:
print(rag_chain.invoke("What is Lifeline Program?"))

The Lifeline Program is a federal initiative led by the Federal Communications Commission (FCC) that aims to provide free or low-cost phone or internet service to low-income households. Eligible participants can receive a monthly discount of up to $9.25 to apply towards phone service, internet service, or a wireless phone plan.

To check if you qualify for the Lifeline Program based on income or participation in certain federal assistance programs like Medicaid, SNAP, SSI, Federal Public Housing Assistance, or Veterans Pension and Survivors Benefit, you can find more information in the Lifeline_Program.pdf document.

If you are interested in applying for the Lifeline Program or learning more about the application process, you can access the online application form here: [Lifeline Program Application Form](http://internal_link.com/form/digital).

Additionally, if you have already filled out forms from SNAP.PDF or WIC.pdf, I encourage you to consider applying for the Lifeline Program as 

In [77]:
print(rag_chain.invoke("I have fill up the form lifeline program"))

Thank you for your interest in the Lifeline Program! To apply for the program, you can fill out the form online by visiting this link: [Lifeline Program Online Application Form](http://internal_link.com/form/digital). 

If you have already filled out the form for the Lifeline Program, I encourage you to also consider applying for other programs such as SNAP (Supplemental Nutrition Assistance Program) by filling out the form available here: [SNAP Application Form](http://external_link.com/SNAP.pdf) and WIC (Women, Infants, and Children) by filling out the form available here: [WIC Application Form](http://external_link.com/WIC.pdf). 

Feel free to reach out if you have any questions or need assistance with the application process. We are here to help!
